# Inception in RAIL - Demo

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 
import rail
import qp
from rail.core.data import TableHandle
from rail.core.stage import RailStage

from rail.estimation.algos.rail_inception import Inform_Inception, Inception

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

### Preparation

To be understood by the estimator, the input data has to be reorganised as a 2d-array, so for each galaxy the first value will be the redshift and all the following is the flatten image.

In [ ]:
img = np.load('/global/cfs/cdirs/lsst/groups/PZ/valentin_image_data_temp/img_30k.npy')
z = np.load('/global/cfs/cdirs/lsst/groups/PZ/valentin_image_data_temp/z_30k.npy')

In [ ]:
img = img.reshape((-1, 64*64*5))
img.shape

In [ ]:
# Check the unflatten step
img_0 = img[1].reshape((64, 64, 5))
plt.imshow(img_0[..., 2:5]);

In [ ]:
z = z.reshape((-1,1))
train_data = np.concatenate((z[:20000], img[:20000]), axis = 1)
test_data = np.concatenate((z[20000:], img[20000:]), axis = 1)

train_data.shape

In [ ]:
training = DS.add_data("training", train_data, TableHandle)
testing = DS.add_data("testing", test_data, TableHandle)

### Training

In [ ]:
inc_dict = dict(trainfrac=0.75,
                epoch=25,
                hdf5_groupname='photometry')

In [ ]:
Inform = Inform_Inception.make_stage(name='Inform_Inception', model='demo_inc.pkl', **inc_dict)

In [ ]:
Inform.inform(training)

### Results

In [ ]:
Inception_model = Inception.make_stage(name='Inception', hdf5_groupname='photometry',
                                       model=Inform.get_handle('model'))

In [ ]:
results = Inception_model.estimate(testing)